In [7]:
# If you haven't already created an activated a virtual environment for this notebook, run this cell
!python3 -m venv venv
!source venv/bin/activate
!pip install -r requirements.txt

In [8]:
import scrapy
import os
current_dir = os.path.abspath('')
url = os.path.join(current_dir, "html/1992_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
with open(url) as _f:
    url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')

In [9]:
# Make sure that the interesting data is available 
table = response.xpath('//table')[1].xpath('tbody')
for tr in table.xpath('tr'):
    print(tr.xpath('td/b/text()').extract()[0],
          tr.xpath('td/a/text()').extract()[0]
    )

Gold Steve Smith
Silver Tim Forsyth
Bronze Takahiro Kimino
